In [1]:
!pip install mlflow

In [2]:
# Test mlflow

import mlflow

mlflow.set_tracking_uri("http://ec2-54-159-96-0.compute-1.amazonaws.com:5000/")

with mlflow.start_run():
    mlflow.log_param("param1", 15)
    mlflow.log_metric("metric1", 0.89)

2025/09/25 13:16:04 INFO mlflow.tracking._tracking_service.client: 🏃 View run caring-stoat-424 at: http://ec2-54-159-96-0.compute-1.amazonaws.com:5000/#/experiments/0/runs/e4682e5339a84cda8abec4edc912f6be.
2025/09/25 13:16:04 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://ec2-54-159-96-0.compute-1.amazonaws.com:5000/#/experiments/0.


In [3]:
#creating baseline model

import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/Himanshu-1703/reddit-sentiment-analysis/refs/heads/main/data/reddit.csv')
df.head()

,clean_comment,category
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1


In [5]:
df.dropna(inplace=True)

In [6]:
df.drop_duplicates(inplace=True)

In [7]:
df = df[~(df['clean_comment'].str.strip() == '')]

In [8]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [9]:
# Ensure necessary NLTK data is downloaded
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
# Define the preprocessing function
def preprocess_comment(comment):
    # Convert to lowercase
    comment = comment.lower()

    # Remove trailing and leading whitespaces
    comment = comment.strip()

    # Remove newline characters
    comment = re.sub(r'\n', ' ', comment)

    # Remove non-alphanumeric characters, except punctuation
    comment = re.sub(r'[^A-Za-z0-9\s!?.,]', '', comment)

    # Remove stopwords but retain important ones for sentiment analysis
    stop_words = set(stopwords.words('english')) - {'not', 'but', 'however', 'no', 'yet'}
    comment = ' '.join([word for word in comment.split() if word not in stop_words])

    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    comment = ' '.join([lemmatizer.lemmatize(word) for word in comment.split()])

    return comment

In [11]:
# Apply the preprocessing function to the 'clean_comment' column
df['clean_comment'] = df['clean_comment'].apply(preprocess_comment)

In [12]:
df.head()

,clean_comment,category
0,family mormon never tried explain still stare ...,1
1,buddhism much lot compatible christianity espe...,1
2,seriously say thing first get complex explain ...,-1
3,learned want teach different focus goal not wr...,0
4,benefit may want read living buddha living chr...,1


In [13]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_predict, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
# Step 1: Vectorize the comments using Bag of Words (CountVectorizer)
vectorizer = CountVectorizer(max_features=10000)  # Bag of Words model with a limit of 1000 features

In [15]:
X = vectorizer.fit_transform(df['clean_comment']).toarray()
y = df['category']  # Assuming 'sentiment' is the target variable (0 or 1 for binary classification)

In [16]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [17]:
X.shape

(36793, 10000)

In [19]:
y

0        1
1        1
2       -1
3        0
4        1
        ..
37244    0
37245    1
37246    0
37247    1
37248    0
Name: category, Length: 36793, dtype: int64

In [20]:
y.shape

(36793,)

In [21]:
# Step 2: Set up the MLflow tracking server
mlflow.set_tracking_uri("http://ec2-54-159-96-0.compute-1.amazonaws.com:5000/")

In [22]:
# Set or create an experiment
mlflow.set_experiment("RF Baseline")

2025/09/25 13:30:08 INFO mlflow.tracking.fluent: Experiment with name 'RF Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow-bucket-102/368303739779906751', creation_time=1758796208700, experiment_id='368303739779906751', last_update_time=1758796208700, lifecycle_stage='active', name='RF Baseline', tags={}>

In [23]:
!pip install boto3

In [24]:
!pip install awscli

   ---------------------------------------- 0.0/4.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.7 MB ? eta -:--:--
   -- ------------------------------------- 0.3/4.7 MB ? eta -:--:--
   -- ------------------------------------- 0.3/4.7 MB ? eta -:--:--
   -- ------------------------------------- 0.3/4.7 MB ? eta -:--:--
   -- ------------------------------------- 0.3/4.7 MB ? eta -:--:--
   -- ------------------------------------- 0.3/4.7 MB ? eta -:--:--
   ---- ----------------------------------- 0.5/4.7 MB 242.9 kB/s eta 0:00:18
   ---- ----------------------------------- 0.5/4.7 MB 242.9 kB/s eta 0:00:18
   ---- ----------------------------------- 0.5/4.7 MB 242.9 kB/s eta 0:00:18
   ---- ----------------------------------- 0.5/4.7 MB 242.9 kB/s eta 0:00:18
   ------ --------------------------------- 0.8/4.7 MB 284.3 kB/s e

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.15.2 requires botocore<1.35.37,>=1.35.16, but you have botocore 1.40.38 which is incompatible.
boto3 1.35.36 requires botocore<1.36.0,>=1.35.36, but you have botocore 1.40.38 which is incompatible.
boto3 1.35.36 requires s3transfer<0.11.0,>=0.10.0, but you have s3transfer 0.14.0 which is incompatible.


In [ ]:
!aws configure

^C


File association not found for extension .py


AWS Access Key ID [****************6ZXH]: 


In [32]:
# Step 1: Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Step 2: Define and train a Random Forest baseline model using a simple train-test split
with mlflow.start_run() as run:
    # Tags / description
    mlflow.set_tag("mlflow.runName", "RandomForest_Baseline_TrainTestSplit")
    mlflow.set_tag("experiment_type", "baseline")
    mlflow.set_tag("model_type", "RandomForestClassifier")
    mlflow.set_tag("description", "Baseline RandomForest model for sentiment analysis using Bag of Words (BoW) with a simple train-test split")

    # Vectorizer params
    mlflow.log_param("vectorizer_type", "CountVectorizer")
    mlflow.log_param("vectorizer_max_features", getattr(vectorizer, "max_features", None))

    # RF params
    n_estimators = 200
    max_depth = 15
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)

    # Train
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    model.fit(X_train, y_train)

    # Predict
    y_pred = model.predict(X_test)

    # Metrics
    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", accuracy)

    classification_rep = classification_report(y_test, y_pred, output_dict=True)
    for label, metrics in classification_rep.items():
        if isinstance(metrics, dict):
            for metric, value in metrics.items():
                # MLflow metrics must be numeric; skip support counts that might be ints (ok) or NaN
                if isinstance(value, (int, float)) and value == value:  # not NaN
                    mlflow.log_metric(f"{label}_{metric}", float(value))

    # Confusion matrix -> log figure directly (no filesystem path hassles)
    conf_matrix = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title("Confusion Matrix")
    plt.tight_layout()
    mlflow.log_figure(plt.gcf(), "confusion_matrix.png")
    plt.close()

    # Log the model
    mlflow.sklearn.log_model(model, artifact_path="random_forest_model")

    # Optionally log the dataset content as a text artifact (no local file needed)
    mlflow.log_text(df.to_csv(index=False), "dataset.csv")

# Display final accuracy
print(f"Accuracy: {accuracy:.4f}")


2025/09/25 14:58:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/09/25 14:59:16 INFO mlflow.tracking._tracking_service.client: 🏃 View run RandomForest_Baseline_TrainTestSplit at: http://ec2-54-159-96-0.compute-1.amazonaws.com:5000/#/experiments/368303739779906751/runs/aace497d811b479a9f24e99618fcf43c.
2025/09/25 14:59:16 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://ec2-54-159-96-0.compute-1.amazonaws.com:5000/#/experiments/368303739779906751.


Accuracy: 0.6479


In [33]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       1.00      0.01      0.02      1650
           0       0.68      0.81      0.74      2555
           1       0.62      0.85      0.72      3154

    accuracy                           0.65      7359
   macro avg       0.77      0.56      0.49      7359
weighted avg       0.73      0.65      0.57      7359



In [34]:
df.to_csv('reddit_preprocessing.csv', index=False)

In [35]:
pd.read_csv('reddit_preprocessing.csv').head()

,clean_comment,category
0,family mormon never tried explain still stare ...,1
1,buddhism much lot compatible christianity espe...,1
2,seriously say thing first get complex explain ...,-1
3,learned want teach different focus goal not wr...,0
4,benefit may want read living buddha living chr...,1
